In [9]:
!pip install plotly

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import networkx as nx

%matplotlib inline

### Read csv data

In [182]:
data_df = pd.read_csv('data.csv', encoding = "ISO-8859-1")
data_df.head()

,id,scientist,Vietnamese Name,field.1,field.2,au.key,au.solo,au.coll,au.uniq,au.vn,...,age,restime,sex,ceb,affil1,affil2,Full affiliation name,prov/coun,region,intexp
0,s001,nguyenngocanh,Nguyen Ngoc Anh,econ,manag,3,0,3,11,9,...,46,20,M,1.0,Depocen,NaN,Dev and Pol Res Center,HN,North,NaN
1,s002,phamntloan,Pham N T Loan,econ,manag,2,1,1,4,1,...,30,4,F,2.0,BUH,NaN,Banking Univ HCM,HCM,South,NaN
2,s003,vuongquanhoang,Vuong Quan Hoang,econ,health,23,11,2,12,10,...,46,20,M,3.0,FPTU,FSB,FBT School of Business,HN,North,NaN
3,s004,nguyenvietcuong,Nguyen Viet Cuong,econ,pol,37,23,14,26,15,...,41,15,M,4.0,NEU,IPMM,"Inst Pub Pol & Manag, Natl Econ Univ",HN,North,NaN
4,s005,giangthanhlong,Giang Thanh Long,pol,econ,11,4,7,12,8,...,42,15,M,4.0,NEU,IPMM,Natl Econ Univ,HN,North,NaN


In [184]:
link_df = pd.read_csv('link_data.csv', encoding='utf-8')
link_df.head()

,from,to,weight,type
0,s001,s073,1,colla
1,s001,s072,1,colla
2,s001,s054,1,colla
3,s004,s046,1,colla
4,s004,s080,1,colla


### Create network graph

In [209]:
def get_name(id_):
    return data_df[data_df.id == id_]['Vietnamese Name'].item().strip()


social_sciences = nx.Graph()

unused_nodes = set()

for id_, name, n_article in data_df[['id', 'Vietnamese Name', 'ttlitems']].values.tolist():
    if n_article >= 5:
        social_sciences.add_node(name.strip(), size = n_article)
    else:
        unused_nodes.add(id_)
    
for f, t, w in link_df[['from', 'to', 'weight']].values.tolist():
    if f not in unused_nodes and t not in unused_nodes:
        social_sciences.add_edge(get_name(f), get_name(t), weight=w)
        
# Get positions for the nodes in G
pos_ = nx.spring_layout(social_sciences)

In [211]:
# Custom function to create an edge between node x and node y, with a given text and width
def make_edge(x, y, text, width):
    return  go.Scatter(x         = x,
                       y         = y,
                       line      = dict(width = width,
                                   color = 'cornflowerblue'),
                       hoverinfo = 'text',
                       text      = ([text]),
                       mode      = 'lines')

# For each edge, make an edge_trace, append to list
edge_trace = []
for edge in social_sciences.edges():
    if social_sciences.edges()[edge]['weight'] > 0:
        char_1 = edge[0]
        char_2 = edge[1]
        x0, y0 = pos_[char_1]
        x1, y1 = pos_[char_2]
        text   = char_1 + '--' + char_2 + ': ' + str(social_sciences.edges()[edge]['weight'])
        trace  = make_edge([x0, x1, None], [y0, y1, None], text, 
                           width = social_sciences.edges()[edge]['weight'])
        edge_trace.append(trace)

In [212]:
node_trace = go.Scatter(x         = [],
                        y         = [],
                        text      = [],
                        textposition = "top center",
                        textfont_size = 10,
                        mode      = 'markers+text',
                        hoverinfo = 'text',
                        marker    = dict(color = [], size = [], line  = None))

# For each node in graph, get the position and size and add to the node_trace
for node in social_sciences.nodes():
    x, y = pos_[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['marker']['color'] += tuple(['cornflowerblue'])
    node_trace['marker']['size'] += tuple([social_sciences.nodes()[node]['size']])
    node_trace['text'] += tuple(['<b>' + node + '</b>'])

### Visualize graph

In [215]:
# Customize layout
layout = go.Layout(
    title='Mạng các trích dẫn',
    titlefont_size=16,
    showlegend=False,
    hovermode='closest',
    margin=dict(b=20,l=5,r=5,t=40),
    paper_bgcolor='rgba(0,0,0,0)', # transparent background
    plot_bgcolor='rgba(0,0,0,0)', # transparent 2nd background
    xaxis =  {'showgrid': False, 'zeroline': False}, # no gridlines
    yaxis = {'showgrid': False, 'zeroline': False}, # no gridlines
)

# Create figure
fig = go.Figure(layout = layout)

# Add all edge traces
for trace in edge_trace:
    fig.add_trace(trace)
    
# Add node trace
fig.add_trace(node_trace)

# Remove legend
fig.update_layout(showlegend = False)

# Remove tick labels
fig.update_xaxes(showticklabels = False)
fig.update_yaxes(showticklabels = False)

# Show figure
fig.show()